TalkingData AdTracking Fraud Detection
-----------------------
*Luiz Sampaio e Rodrigo Escorcio* - Prof: Renato Souza

Objetiva-se fazer uma análise preditiva dos dados da competição "TalkingData AdTracking Fraud Detection Challenge" do Kaggle (https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection)

Os dados consistem de informações para cada click, como: IP, o aplicativo que estava sendo utilizado, o aparelho utilizado, a versão do sistema operacional do aparelho, o horário do click, se o usuário de fato fez o download e que horas foi feito.

In [29]:
# Importando bibliotecas

import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import pathlib
import csv
import matplotlib.pyplot as plt

In [30]:
csvtrain = "train.csv"
csvtest = "test.csv" #declarar (utilizamos uma parcela dos dados, já que nao foi possivel rodar na maquina virtual) 

In [31]:
df_train = pd.read_csv(csvtrain) #importar dados
df_test = pd.read_csv(csvtest)

df_train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,70335,3,1,8,130,07/11/2017,04:21:42,0
1,357443,15,1,13,265,09/11/2017,02:02:34,0
2,37515,13,1,37,477,06/11/2017,22:55:26,0
3,3219,9,1,19,215,07/11/2017,04:00:26,0
4,61800,15,1,9,130,09/11/2017,03:19:46,0


In [32]:
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
lb = LabelBinarizer()
mlb = MultiLabelBinarizer()

In [33]:
num = df_train['app'].values
N = np.shape(np.unique(num))[0] 

lista = []
for i in range(N):
    lista.append('app'+str(i))
    
apps = pd.DataFrame(lb.fit_transform(df_train['app'].values),columns=lista)
df_train = pd.concat([df_train,apps], axis=1)


In [34]:
num = df_train['device'].values
M = np.shape(np.unique(num))[0] 

lista = []
for i in range(M):
    lista.append('device'+str(i))

devices = pd.DataFrame(lb.fit_transform(df_train['device'].values),columns=lista)
df_train = pd.concat([df_train,devices], axis=1)

In [35]:
num = df_train['os'].values
O = np.shape(np.unique(num))[0] 

lista = []
for i in range(O):
    lista.append('os'+str(i))

oss = pd.DataFrame(lb.fit_transform(df_train['os'].values),columns=lista)
df_train = pd.concat([df_train,oss], axis=1)

In [38]:
num = df_train['channel'].values
P = np.shape(np.unique(num))[0] 

lista = []
for i in range(P):
    lista.append('channel'+str(i))

channels = pd.DataFrame(lb.fit_transform(df_train['channel'].values),columns=lista)
df_train = pd.concat([df_train,channels], axis=1)

df_train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,app0,app1,...,channel148,channel149,channel150,channel151,channel152,channel153,channel154,channel155,channel156,channel157
0,70335,3,1,8,130,07/11/2017,04:21:42,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,357443,15,1,13,265,09/11/2017,02:02:34,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,37515,13,1,37,477,06/11/2017,22:55:26,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3219,9,1,19,215,07/11/2017,04:00:26,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,61800,15,1,9,130,09/11/2017,03:19:46,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
tmp = df_train['attributed_time']
tmp2 = tmp.tolist()
lista = []
for time in tmp2:
    stime = time.split(":")
    lista.append((int(stime[0])*3600+int(stime[1])*60+int(stime[2]))/86400.0)

times = pd.DataFrame(lista,columns=["attr_time"])
df_train = pd.concat([df_train,times], axis=1)

tmp3 = df_train['is_attributed']
tmp4 = tmp3.tolist()
lista = []
for time in tmp4:
    print(time)
    stime = int(time)
    lista.append(stime)

times = pd.DataFrame(lista,columns=["is_attr"])
df_train = pd.concat([df_train,times], axis=1)

df_train2 = df_train.drop(['ip','app','device','os','channel','click_time','attributed_time'], axis=1)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
07/11/2017


ValueError: invalid literal for int() with base 10: '07/11/2017'

In [ ]:
df_train2.head()

In [ ]:
train_data = df_train2.values
train_data